# Car following forecasting

This tutorial is an introduction to time series forecasting using TensorFlow. It builds a few different styles of models including Convolutional and Recurrent Neural Networks (CNNs and RNNs).

This is covered in two main parts, with subsections: 

* Forecast for a single time step:
  * A single feature.
  * All features.
* Forecast multiple steps:
  * Single-shot: Make the predictions all at once.
  * Autoregressive: Make one prediction at a time and feed the output back to the model.

## Setup

In [1]:
import os, sys
import datetime

import IPython
import IPython.display
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
from tqdm import tqdm
import pickle

mpl.rcParams['figure.figsize'] = (8, 6)
mpl.rcParams['axes.grid'] = False

## The driving trajectory dataset

This tutorial uses a <a href="https://www.highd-dataset.com/" class="external">driving trajectory dataset</a> recorded by the <a href="https://www.rwth-aachen.de/go/id/a/?lidx=1" class="external">RWTH Aachen University</a>.

According to the <a href="https://www.highd-dataset.com/format" class="external">dataset format</a>, traffic was recorded at 6 different locations and includes more than 110,500 vehicles. Each location's recorded distance is 400-420 meters. Each vehicle's trajectory, including vehicle type, size and manoeuvres, is extracted. These were collected every 0.04 seconds (i.e., the frame rate is 25 fps). For efficiency, we will use only the data collected at location 2. 

In [2]:
indir = './outputs/tracks-slurm/'
outdir = './outputs/unina-cf-nns/'

df_meta = pd.read_pickle(indir + 'meta.pkl')
recIdSet = list(df_meta.loc[df_meta['locationId']==2, 'recordingId'].unique())
recIdSet

[1, 2, 3]

In [3]:
df = pd.DataFrame()
classId = 0 # car-0, truck-1
veh_class = 'car' if classId==0 else 'truck'

for i in recIdSet:
    idx_str = '{0:02}'.format(i)
    df_ = pd.read_pickle(indir + idx_str + '_data.pkl')
    df_.insert(loc=0, column='recordingId', value=i)
    df = pd.concat([df, df_])
df =  df.rename(columns={'id': 'vehicleId'})
df = df.loc[
    (df['class'] == classId) &   # 0-car, 1-truck
    (df['numFrames'] >= 200) & 
    (df['numFrames'] <= 500)].reset_index(drop=True)
# df.to_pickle(outdir + 'data.pkl')
df.head(2)

,recordingId,vehicleId,width,height,initialFrame,finalFrame,numFrames,class,drivingDirection,traveledDistance,...,Left_Fol_X,Left_Fol_Speed,Right_Pre_X,Right_Pre_Speed,Right_Al_X,Right_Al_Speed,Right_Fol_X,Right_Fol_Speed,traffic_density,traffic_speed
0,1,9.0,4.85,2.02,1.0,223.0,223.0,0.0,1.0,323.25,...,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[5.89622641509434, 7.0754716981132075, 7.07547...","[36.922000000000004, 37.19833333333334, 37.208..."
1,1,11.0,4.14,1.92,1.0,235.0,235.0,0.0,2.0,231.76,...,"[29.960000000000008, 29.659999999999997, 29.38...","[32.59, 32.6, 32.61, 32.62, 32.64, 32.65, 32.6...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[8.254716981132075, 9.433962264150944, 9.43396...","[30.92142857142857, 32.245000000000005, 32.25,..."


### 4 types of input data
- Longitudinal + lateral + traffic (27)
- Longitudinal + lateral (25)
- Longitudinal + traffic (11)
- Longitudinal only (9)

In [4]:
df_traj = df.loc[:, 'frame':'traffic_speed']
col_full = df_traj.columns.values.tolist()

col_drop = ['frame', 'precedingId', 'followingId', 'leftFollowingId', 'leftAlongsideId',
            'leftPrecedingId', 'rightFollowingId', 'rightAlongsideId', 'rightPrecedingId']
# Drop lateral information
# col_drop += ['y', 'yVelocity', 'yAcceleration', 'laneId', 
             # 'Left_Pre_X', 'Left_Pre_Speed', 'Left_Al_X', 'Left_Al_Speed', 'Left_Fol_X', 'Left_Fol_Speed',
             # 'Right_Pre_X', 'Right_Pre_Speed', 'Right_Al_X', 'Right_Al_Speed', 'Right_Fol_X', 'Right_Fol_Speed']
# Drop traffic information
# col_drop += ['traffic_density', 'traffic_speed']

col_inputs = [x for x in col_full if x not in col_drop]
len(col_inputs), col_inputs

(27,
 ['x',
  'y',
  'xVelocity',
  'yVelocity',
  'xAcceleration',
  'yAcceleration',
  'frontSightDistance',
  'backSightDistance',
  'thw',
  'ttc',
  'dhw',
  'precedingXVelocity',
  'laneId',
  'Left_Pre_X',
  'Left_Pre_Speed',
  'Left_Al_X',
  'Left_Al_Speed',
  'Left_Fol_X',
  'Left_Fol_Speed',
  'Right_Pre_X',
  'Right_Pre_Speed',
  'Right_Al_X',
  'Right_Al_Speed',
  'Right_Fol_X',
  'Right_Fol_Speed',
  'traffic_density',
  'traffic_speed'])

In [5]:
flatten = {}
for col in col_inputs:
    arr = df_traj[col].to_numpy()
    flatten[col] = np.concatenate(arr).ravel()
df_flatten = pd.DataFrame.from_dict(flatten)
df_des = df_flatten.describe().transpose()
scndir = outdir + f'{len(col_inputs)}-{veh_class}/'
os.makedirs(scndir, exist_ok=True)
df_des.to_csv(scndir + 'data_des.csv')
df_traj.to_csv(scndir + 'data_traj.csv', index=False)
df_flatten.to_csv(scndir + 'data_flatten.csv', index=False)
df_des.head(2)

,count,mean,std,min,25%,50%,75%,max
x,738620.0,204.208294,117.504452,-13.47,102.46,203.74,305.71,414.68
y,738620.0,16.301617,5.740969,7.95,12.73,13.43,21.89,27.49


### WindowGenerator class

In [ ]:
class WindowGenerator():
    def __init__(self, input_width, label_width, shift, batch_size=32,
                 df=None, ds=None, label_columns=None, column_indices=None):
        self.batch_size = batch_size

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
            self.label_columns_indices = {name: i for i, name in
                                          enumerate(label_columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]
        
        # Store the raw data.
        if df is None:
            assert ds is not None and column_indices is not None
            self.ds = ds
            self.column_indices = column_indices
        else:
            self.df = df
            self.column_indices = {name: i for i, name in
                       enumerate(self.df.columns)}
            self.ds = self.make_dataset(self.df)
    
    
    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])
    
    
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=self.batch_size,)

        ds = ds.map(self.split_window)

        return ds
    
    
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:, :, self.column_indices[name]] for name in self.label_columns],
                axis=-1)

        # Slicing doesn't preserve static shape information, so set the shapes
        # manually. This way the `tf.data.Datasets` are easier to inspect.
        inputs.set_shape([None, self.input_width, None])
        labels.set_shape([None, self.label_width, None])

        return inputs, labels
    
    
    def plot(self, model=None, plot_col='xAcceleration', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(12, 8))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots, len(inputs))
        for n in range(max_n):
            plt.subplot(max_n, 1, n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n, :, plot_col_index],
                     label='Inputs', marker='.', zorder=-10)

            if self.label_columns:
                label_col_index = self.label_columns_indices.get(plot_col, None)
            else:
                label_col_index = plot_col_index

            if label_col_index is None:
                continue

            plt.scatter(self.label_indices, labels[n, :, label_col_index],
                        edgecolors='k', label='Labels', c='#2ca02c', s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices, predictions[n, :, label_col_index],
                            marker='X', edgecolors='k', label='Predictions',
                            c='#ff7f0e', s=64)

            if n == 0:
                plt.legend()

        plt.xlabel('Step')
    
    
    def get_dataset_partitions_tf(self, ds_size, train_split=0.7, val_split=0.2, test_split=0.1, shuffle=True, shuffle_size=1000):
        
        assert (train_split + test_split + val_split) == 1
        
        # ds_size = len(list(self.ds))
        
        if shuffle:
            # Specify seed to always have the same split distribution between runs
            ds = self.ds.shuffle(shuffle_size, seed=12)

        train_size = int(train_split * ds_size)
        val_size = int(val_split * ds_size)

        train = ds.take(train_size)
        val = ds.skip(train_size).take(val_size)
        test = ds.skip(train_size).skip(val_size)

        return train, val, test
    
    
    @property
    def example(self):
        """Get and cache an example batch of `inputs, labels` for plotting."""
        result = getattr(self, '_example', None)
        if result is None:
            # No example batch was found, so get one from the `.train` dataset
            result = next(iter(self.ds))
            # And cache it for next time
            self._example = result
        return result

In [ ]:
# 10_10_10, 10_5_5, 5_1_1
input_width = 10
label_width = 10
shift = 10

casedir = outdir+f'{len(col_inputs)}-{veh_class}/{input_width}_{label_width}_{shift}/'

dsdir = casedir+'saved_data'
print(casedir)

### Generate and save dataset
This tutorial will just deal with **short-term predictions**, so start by sub-sampling the data from 0.04-second (1/25) intervals to 0.2-second intervals:

In [ ]:
df_traj = df.loc[:, col_inputs]
w = None
ds_size = 0
for idx, row in tqdm(df_traj.iterrows(), total=df_traj.shape[0], mininterval=5):
    df_ = pd.DataFrame()
    for col_name, col_val in row.iteritems():
        df_[col_name] = col_val
    df_ = df_[4::5]

    df_ = (df_ - df_des['mean']) / df_des['std']
    # df_ = (df_ - df_des['min']) / (df_des['max'] - df_des['min'])
    w_ = WindowGenerator(input_width=input_width, label_width=label_width, shift=shift, df=df_,
                         label_columns=['xAcceleration'])
    ds_size += len(list(w_.ds))
    if w == None:
        w = w_
    else:
        w.ds = w.ds.concatenate(w_.ds)
    
    if idx==50:
        break

print(w)
tf.data.experimental.save(w.ds, dsdir)
with open(casedir+'/ds_size.txt', 'w') as f:
    f.write('%d' % ds_size)
with open(casedir+'/column_indices.pkl', 'wb') as f:
    pickle.dump(w.column_indices, f)
print('\n Data saved!')

### Load saved dataset

In [ ]:
ds_load = tf.data.experimental.load(dsdir)
with open(casedir+'/ds_size.txt', 'r') as f:
    ds_size = int(f.readline())
with open(casedir+'/column_indices.pkl', 'rb') as f:
    cols_load = pickle.load(f)

w = WindowGenerator(input_width=input_width, label_width=label_width, shift=shift, ds=ds_load,
                    label_columns=['xAcceleration'], column_indices=cols_load)
print('\n Data loaded!')

In [ ]:
w.plot()

In [ ]:
w.train, w.val, w.test = w.get_dataset_partitions_tf(ds_size)
print('\n Data splitted!')

In [ ]:
multi_val_performance = {}
multi_performance = {}

def model_fit(model, window, max_epochs, patience=2):
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                      patience=patience,
                                                      mode='min')
    history = model.fit(window.train, epochs=max_epochs,
                        validation_data=window.val,
                        verbose=1,
                        callbacks=[early_stopping])
    return history, model

In [ ]:
num_features = len(w.column_indices)
OUT_STEPS = w.label_width
modeldir = casedir+'/saved_model'
MAX_EPOCHS = 30

if os.path.exists(modeldir+'/my_model'):
    multi_lstm_model = tf.keras.models.load_model(modeldir+'/my_model')
    print('\n Model loaded!')
else:
    multi_lstm_model = tf.keras.Sequential([
        tf.keras.layers.LSTM(32, return_sequences=False),
        tf.keras.layers.Dense(OUT_STEPS*num_features,
                              kernel_initializer=tf.initializers.zeros()),
        tf.keras.layers.Reshape([OUT_STEPS, num_features])
    ])
    multi_lstm_model.compile(loss=tf.keras.losses.MeanSquaredError(),
              optimizer=tf.keras.optimizers.Adam(),
              metrics=[tf.keras.metrics.MeanAbsoluteError()])
    print('\n Model created!')
    
history, model = model_fit(multi_lstm_model, w, MAX_EPOCHS)

# IPython.display.clear_output()

multi_val_performance['LSTM'] = model.evaluate(w.val)
multi_performance['LSTM'] = model.evaluate(w.test, verbose=2)

model.summary()
model.save(modeldir+'/my_model')
print('\n Model saved!')
w.plot(model)